In [ ]:

import matplotlib
matplotlib.use("PDF")
import pyfits as pf
from pylab import *
import sys, glob
from scipy import stats

thedir='/sandbox/lsst/lsst/GUI/20161010_029-cte/'
%cd $thedir

#%matplotlib inline

In [ ]:
#files = glob.glob('ITL-3800C-029_flat_light_004_20161010??????.fits')
files = glob.glob('ITL-3800C-029_flat_light_002_2016101010????.fits')
print files


In [ ]:
xstart = 505
xstop = 542
ov_start = 512
ystart = 500
ystop = 1500
xaxis = linspace(xstart,xstop-1,xstop-xstart)
for file in files:
    hdulist = pf.open(file, mode='readonly', do_not_scale_image_data=True)
    for segment in range(16):#[8]:
        try:
            imhdr=hdulist[segment+1].header
            extname = imhdr['EXTNAME']
            figure()
            suptitle("CTI - %s"%extname)
            data = array(hdulist[segment+1].data + 32768, dtype = int32)
            flat_overscan = array(data[:,xstop-8:xstop].sum(axis = 1) / 8.0)
            cte_data = ((transpose(transpose(data) - flat_overscan))[ystart:ystop,:].mean(axis=0))[xstart:xstop]
            cte_std = ((transpose(transpose(data) - flat_overscan))[ystart:ystop,:].std(axis=0) / sqrt(float(ystop-ystart)))[xstart:xstop]
            cti = (transpose(transpose(data) - flat_overscan))[ystart:ystop,ov_start].mean()\
            / (transpose(transpose(data) - flat_overscan))[ystart:ystop,ov_start-1].mean() / ov_start
            #print (transpose(transpose(data) - flat_overscan))[ystart:ystop,ov_start].mean()
            #print (transpose(transpose(data) - flat_overscan))[ystart:ystop,ov_start-1].mean()
            #print extname, cti
            #print cte_data
            #sys.exit()
            scatter(xaxis, cte_data, color = 'blue')
            errorbar(xaxis, cte_data, yerr=cte_std)
            text(xstop - 20, 20000, "CTI = %.3E"%cti)
            text(501,20000,"Flat Level",color = 'blue')
            slope1, intercept1, r_value, p_value, std_err = stats.linregress(xaxis[7:9],log10(cte_data[7:9]))
            slope2, intercept2, r_value, p_value, std_err = stats.linregress(xaxis[9:13],log10(cte_data[9:13]))
            slope3, intercept3, r_value, p_value, std_err = stats.linregress(xaxis[13:20],log10(cte_data[13:20]))
            x1 = linspace(xaxis[5],xaxis[10],6)
            y1 = 10**(intercept1 + slope1 * x1)
            plot(x1,y1,color = 'red')
            text(515,10000,"Slope1 = %.6f"%(10**(slope1)), color = 'red')
            x2 = linspace(xaxis[7],xaxis[16],10)
            y2 = 10**(intercept2 + slope2 * x2)
            plot(x2,y2,color = 'green')
            text(515,1000,"Slope2 = %.6f"%(10**(slope2)), color = 'green')
            x3 = linspace(xaxis[5],xaxis[22],18)
            y3 = 10**(intercept3 + slope3 * x3)
            plot(x3,y3,color = 'magenta')
            text(515,100,"Slope3 = %.6f"%(10**(slope3)), color = 'magenta')
            yscale('log',nonposy='clip')
            ylim(.01, 100000)
            xlabel("X (First overscan = 512)")
            ylabel("Signal(ADU)")
            savefig("Serial_CTE_Low_11Oct16_%s.pdf"%extname)
            print "Segment %s done"%extname
        except:
            continue
